In [8]:
import numpy as np 

fracture_train = np.load('./npy/fracture_train.npy', allow_pickle=True)

print(fracture_train.shape)

In [9]:
from __future__ import print_function, division

from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K
import matplotlib.pyplot as plt
import sys
import numpy as np

In [10]:
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss, averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss, self.wasserstein_loss, partial_gp_loss],
                                  optimizer=optimizer,
                                  loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256 * 16 * 16, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((16, 16, 256)))
        model.add(UpSampling2D())
        
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())

        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())

        model.add(Conv2D(32, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, train_data, epochs, batch_size, sample_interval=50):

        # Load the dataset
        X_train = fracture_train

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                # Sample generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                # Train the critic
                d_loss = self.critic_model.train_on_batch([imgs, noise], [valid, fake, dummy])

            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.generator_model.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/whole_%d.png" % epoch)
        plt.close()

In [11]:
wgan = WGANGP()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 65536)             6619136   
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 256)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 32, 32, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       524416    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 128)       0         
_______________________________________

In [12]:
# Wgan 학습
wgan.train(facture_train, epochs=15000, batch_size=32, sample_interval=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/hiwony/.local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/home/hiwony/.local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.563157] [G loss: -0.422373]


/home/hiwony/.local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

14209 [D loss: -0.725533] [G loss: -0.131368]
14210 [D loss: -0.744761] [G loss: -0.115141]
14211 [D loss: -0.660449] [G loss: -0.038891]
14212 [D loss: -0.724624] [G loss: -0.076145]
14213 [D loss: -0.675539] [G loss: -0.155180]
14214 [D loss: -0.645202] [G loss: -0.078930]
14215 [D loss: -0.724359] [G loss: -0.108361]
14216 [D loss: -0.621572] [G loss: -0.085110]
14217 [D loss: -0.484985] [G loss: -0.101193]
14218 [D loss: -0.743505] [G loss: -0.112679]
14219 [D loss: -0.455944] [G loss: -0.144146]
14220 [D loss: -0.728184] [G loss: -0.151881]
14221 [D loss: -0.588093] [G loss: -0.050867]
14222 [D loss: -0.726146] [G loss: -0.187692]
14223 [D loss: -0.666465] [G loss: -0.095688]
14224 [D loss: -0.555182] [G loss: -0.158485]
14225 [D loss: -0.632309] [G loss: -0.053690]
14226 [D loss: -0.754459] [G loss: -0.121347]
14227 [D loss: -0.804065] [G loss: -0.049185]
14228 [D loss: -0.751774] [G loss: -0.066551]
14229 [D loss: -0.586978] [G loss: -0.188974]
14230 [D loss: -0.706514] [G loss:

14388 [D loss: -0.801763] [G loss: -0.132432]
14389 [D loss: -0.742738] [G loss: -0.171104]
14390 [D loss: -0.795020] [G loss: -0.113180]
14391 [D loss: -0.724980] [G loss: -0.229531]
14392 [D loss: -0.753556] [G loss: -0.181845]
14393 [D loss: -0.716186] [G loss: -0.234821]
14394 [D loss: -0.683390] [G loss: -0.103572]
14395 [D loss: -0.727068] [G loss: -0.151482]
14396 [D loss: -0.766318] [G loss: -0.177164]
14397 [D loss: -0.805682] [G loss: -0.089840]
14398 [D loss: -0.698359] [G loss: -0.239419]
14399 [D loss: -0.787008] [G loss: -0.141143]
14400 [D loss: -0.778190] [G loss: -0.145724]
14401 [D loss: -0.610925] [G loss: -0.192474]
14402 [D loss: -0.703905] [G loss: -0.136015]
14403 [D loss: -0.626903] [G loss: -0.161594]
14404 [D loss: -0.780687] [G loss: -0.154197]
14405 [D loss: -0.754166] [G loss: -0.161167]
14406 [D loss: -0.775840] [G loss: -0.151736]
14407 [D loss: -0.809837] [G loss: -0.121605]
14408 [D loss: -0.689367] [G loss: -0.129285]
14409 [D loss: -0.696909] [G loss:

14567 [D loss: -0.768101] [G loss: -0.088039]
14568 [D loss: -0.675375] [G loss: -0.174258]
14569 [D loss: -0.708604] [G loss: -0.156194]
14570 [D loss: -0.745051] [G loss: -0.112950]
14571 [D loss: -0.605359] [G loss: -0.201527]
14572 [D loss: -0.777749] [G loss: -0.138698]
14573 [D loss: -0.764987] [G loss: -0.146816]
14574 [D loss: -0.797212] [G loss: -0.190818]
14575 [D loss: -0.691124] [G loss: -0.191700]
14576 [D loss: -0.823878] [G loss: -0.102794]
14577 [D loss: -0.780345] [G loss: -0.165929]
14578 [D loss: -0.831367] [G loss: -0.121220]
14579 [D loss: -0.786370] [G loss: -0.121068]
14580 [D loss: -0.677051] [G loss: -0.252127]
14581 [D loss: -0.827245] [G loss: -0.141278]
14582 [D loss: -0.649208] [G loss: -0.231257]
14583 [D loss: -0.808708] [G loss: -0.145574]
14584 [D loss: -0.613599] [G loss: -0.180167]
14585 [D loss: -0.718789] [G loss: -0.231178]
14586 [D loss: -0.742358] [G loss: -0.168262]
14587 [D loss: -0.710020] [G loss: -0.115151]
14588 [D loss: -0.773145] [G loss:

14746 [D loss: -0.755461] [G loss: -0.191968]
14747 [D loss: -0.547100] [G loss: -0.174647]
14748 [D loss: -0.748852] [G loss: -0.172698]
14749 [D loss: -0.698212] [G loss: -0.096535]
14750 [D loss: -0.771121] [G loss: -0.178377]
14751 [D loss: -0.542082] [G loss: -0.166678]
14752 [D loss: -0.797089] [G loss: -0.147318]
14753 [D loss: -0.622483] [G loss: -0.120149]
14754 [D loss: -0.584714] [G loss: -0.249651]
14755 [D loss: -0.724073] [G loss: -0.157356]
14756 [D loss: -0.789767] [G loss: -0.132927]
14757 [D loss: -0.688270] [G loss: -0.164215]
14758 [D loss: -0.707540] [G loss: -0.227868]
14759 [D loss: -0.568619] [G loss: -0.234019]
14760 [D loss: -0.826846] [G loss: -0.108766]
14761 [D loss: -0.802556] [G loss: -0.132879]
14762 [D loss: -0.743576] [G loss: -0.176440]
14763 [D loss: -0.711335] [G loss: -0.232743]
14764 [D loss: -0.752375] [G loss: -0.129331]
14765 [D loss: -0.682974] [G loss: -0.156832]
14766 [D loss: -0.582382] [G loss: -0.106416]
14767 [D loss: -0.844823] [G loss:

14925 [D loss: -0.756550] [G loss: -0.077942]
14926 [D loss: -0.568830] [G loss: -0.189956]
14927 [D loss: -0.725800] [G loss: -0.180798]
14928 [D loss: -0.835236] [G loss: -0.157789]
14929 [D loss: -0.734935] [G loss: -0.180165]
14930 [D loss: -0.691566] [G loss: -0.182884]
14931 [D loss: -0.813089] [G loss: -0.143425]
14932 [D loss: -0.788198] [G loss: -0.169416]
14933 [D loss: -0.672520] [G loss: -0.224334]
14934 [D loss: -0.712627] [G loss: -0.131225]
14935 [D loss: -0.620638] [G loss: -0.185726]
14936 [D loss: -0.742948] [G loss: -0.157961]
14937 [D loss: -0.109175] [G loss: -0.197691]
14938 [D loss: -0.766339] [G loss: -0.183008]
14939 [D loss: -0.669556] [G loss: -0.177223]
14940 [D loss: -0.637474] [G loss: -0.176487]
14941 [D loss: -0.821159] [G loss: -0.139159]
14942 [D loss: -0.679998] [G loss: -0.120680]
14943 [D loss: -0.841593] [G loss: -0.124380]
14944 [D loss: -0.725532] [G loss: -0.187424]
14945 [D loss: -0.783040] [G loss: -0.072782]
14946 [D loss: -0.727809] [G loss:

In [ ]:
wgan.critic.save("./models/gan_d_model.h5")